Data Wrangling is finding, structuring, cleaning and adding to your data, validating and preparing for analysis. I am using the dataset from Stanford's Open Policing Project, so much of the finding/structuring/cleaning/validating was already finished.

To get a sense of the data, I will count rows, count features, detail the type of each feature, signify whether the features have categorical data, and look for missing values.

In [1]:
import pandas as pd

In [2]:
# Read in the file and look at the first five rows
colorado = pd.read_table('CO_cleaned.csv',sep=',')
print(colorado.shape)
colorado.head()

/anaconda2/lib/python2.7/site-packages/IPython/core/interactiveshell.py:2718: DtypeWarning: Columns (19,20,21) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


(2584744, 26)


,id,state,stop_date,stop_time,location_raw,county_name,county_fips,fine_grained_location,police_department,driver_gender,...,search_conducted,search_type_raw,search_type,contraband_found,stop_outcome,is_arrested,officer_id,officer_gender,vehicle_type,out_of_state
0,CO-2010-000001,CO,2010-01-01,00:00,10.0,Arapahoe County,8005.0,289,Golden,F,...,False,NaN,NaN,False,Verbal Warning,False,1024976.0,M,NA NA 0,False
1,CO-2010-000002,CO,2010-01-01,00:01,4.0,El Paso County,8041.0,3,Colorado Springs,F,...,False,NaN,NaN,False,Verbal Warning,False,1069803.0,NaN,NA NA 0,False
2,CO-2010-000003,CO,2010-01-01,00:01,24.0,Garfield County,8045.0,3,Glenwood Springs,M,...,False,NaN,NaN,False,Verbal Warning,False,1198321.0,M,NA NA 0,False
3,CO-2010-000004,CO,2010-01-01,00:02,19.0,Yuma County,8125.0,NaN,Sterling,M,...,False,NaN,NaN,False,Citation,False,1167059.0,NaN,Ford F-250 2003,False
4,CO-2010-000005,CO,2010-01-01,00:02,11.0,Jefferson County,8059.0,NaN,6A,F,...,False,NaN,NaN,False,Written Warning,False,1300790.0,NaN,Oldsmobile Intrigue 1999,False


**id** - The unique ID we assign to each stop. Contains the state and year.

**state** - The two-letter code for the state in which the stop occurred

**stop_date** - The date of the stop, in YYYY-MM-DD format. Some states do not provide the exact stop date: for example, they only provide the year or quarter in which the stop occurred. For these states, stop_date is set to the date at the beginning of the period: for example, January 1 if only year is provided.

**stop_time** - The 24-hour time of the stop, in HH:MM format.

**location_raw** - The original data value from which we compute the county (or comparably granular location) in which the stop occurred. Not in a standardized format across states.

**county_name** - The standardized name of the county in which the stop occurred.

**county_fips** - The standardized 5-digit FIPS code in which the stop occurred.

**district** - In several states (e.g., Illinois) the stop county cannot be inferred, but a comparably granular location can. This comparably granular location is stored in the district column. Most states do not have this column.

**fine_grained_location** - Any higher-resolution data about where the stop occurred: e.g., milepost or address. Not standardized across states.

**police_department** - The police department or agency that made the stop. Not in a standard format across states.

**driver_gender** - The driver’s gender, as recorded by the trooper. M, F, or NA.

**driver_age_raw** - The original data value from which we compute the driver’s age when they were stopped. May be age, birth year, or birth date. Not in a standard format across states.

**driver_age** - The driver’s age when they were stopped. Set to NA if less than 15 or greater than or equal to 100.

**driver_race_raw** - The original data value from which the driver’s standardized race is computed. Not in a standard format across states.

**driver_race** - The standardized driver race. Possible values are White, Black, Hispanic, Asian, Other, and NA, with NA denoting values which are unknown. Asian refers to Asian, Pacific Islander, and Indian. Native Americans/American Indians are included in the "other" category. Anyone with Hispanic ethnicity is classified as Hispanic, regardless of their recorded race.

**violation_raw** - The violation committed by the driver, in the language of the original data. Not in a standard format across states. Some stops have multiple violations.

**violation** - The violation committed by the driver, standardized into categories that are consistent across states.

**search_conducted** - A TRUE/FALSE value indicating whether a search was performed.

**search_type_raw** - The justification for the search, in the language of the original data. NA if no search was performed. Not in a standard format across states. Some states have multiple justifications for a search. 

**search_type** - The normalized justification for the search. Where possible, this is standardized into categories which are consistent across states. For example, if something is clearly a consent search, search_type is referred to as “Consent”. 

**contraband_found** - A TRUE/FALSE value indicating whether a search was performed and contraband was found. FALSE if no search was performed. 

**stop_outcome** - The outcome of the stop. Many states have idiosyncratic outcomes — for example, “CHP 215” in California — so this column is not standardized across states. “Citation” and “Warning” are the values which occur most commonly across states. If the stop has multiple outcomes, the most severe outcome is used. For example, if a stop resulted in a citation and a warning, stop_outcome would be “Citation”. 

**is_arrested** - A TRUE/FALSE value indicating whether an arrest was made.

**officer_id**

**officer_gender**

**vehicle_type**

**out_of_state**

*For several fields (e.g., driver_race) we include a "raw" column which records the original data values from which we infer standardized values. For example, driver_race_raw might be “White Hispanic” which we code as “Hispanic” in the standardized driver_race field. We include the raw columns because our data processing pipeline is extensive, requiring judgment calls and subjective decisions. We aim to make our data processing as transparent as possible. Other analysts may choose to process the raw data differently if their needs or judgments differ.* (https://github.com/5harad/openpolicing/blob/master/DATA-README.md)

In [3]:
# Drop the raw columns and other duplicative columns.
colorado.drop(['state','location_raw','driver_age_raw','driver_race_raw','violation_raw','search_type_raw',
               'county_fips','fine_grained_location'],axis=1, inplace = True)
print(colorado.shape)

(2584744, 18)


In [4]:
# Type of value for each column
colorado.dtypes

id                    object
stop_date             object
stop_time             object
county_name           object
police_department     object
driver_gender         object
driver_age           float64
driver_race           object
violation             object
search_conducted        bool
search_type           object
contraband_found      object
stop_outcome          object
is_arrested           object
officer_id           float64
officer_gender        object
vehicle_type          object
out_of_state          object
dtype: object

In [5]:
# I don't think I have to do this
# Change some columns to strings
#colorado[['id','state','county_name','police_department','driver_gender','driver_race','violation','search_type',
#          'stop_outcome','officer_gender','vehicle_type']] = colorado[['id','state','county_name','police_department',
 #                                                     'driver_gender','driver_race','violation','search_type',
 #                                                     'stop_outcome','officer_gender','vehicle_type']].astype(str)

In [6]:
# Change stop_date and stop_time columns to datetime format
# https://stackoverflow.com/questions/15891038/change-data-type-of-columns-in-pandas
colorado['stop_date'] = pd.to_datetime(colorado['stop_date'])
colorado['stop_time'] = pd.to_datetime(colorado['stop_time'])

In [7]:
# Change three columns to boolean
colorado[['contraband_found','is_arrested','out_of_state']] = colorado[['contraband_found','is_arrested',
                                                                        'out_of_state']].astype(bool)

In [8]:
# Type of value for each column
colorado.dtypes

id                           object
stop_date            datetime64[ns]
stop_time            datetime64[ns]
county_name                  object
police_department            object
driver_gender                object
driver_age                  float64
driver_race                  object
violation                    object
search_conducted               bool
search_type                  object
contraband_found               bool
stop_outcome                 object
is_arrested                    bool
officer_id                  float64
officer_gender               object
vehicle_type                 object
out_of_state                   bool
dtype: object

Almost all columns are categorical. The exceptions are the 'id' column, and the stop_date and stop_time.

In [9]:
# Count the number of null values in each column. In total there are 2,584,744 rows.
colorado.isnull().sum()

id                         0
stop_date                  3
stop_time                  0
county_name               27
police_department       4688
driver_gender         368416
driver_age            396639
driver_race           399256
violation             458176
search_conducted           0
search_type          2575920
contraband_found           0
stop_outcome         1288442
is_arrested                0
officer_id                 2
officer_gender       1594782
vehicle_type               0
out_of_state               0
dtype: int64

In [10]:
colorado['stop_outcome'].unique()

array(['Verbal Warning', 'Citation', 'Written Warning', 'Arrest', nan], dtype=object)

With so many null values in the stop_outcome column (1,288,442), almost half of the total values, we were interested to see what the unique values were. There are only four other values, which is not going to be a problem.

In [12]:
# Count the number of unique values in each column
colorado.apply(lambda x: len(x.unique()))

id                   2584744
stop_date               2289
stop_time               1440
county_name               65
police_department         54
driver_gender              3
driver_age                86
driver_race                6
violation               1953
search_conducted           2
search_type                4
contraband_found           2
stop_outcome               5
is_arrested                2
officer_id           2564338
officer_gender             6
vehicle_type          154137
out_of_state               2
dtype: int64

The 'violation' and the 'vehicle_type' have quite a lot of unique values. We will still try and expand the categorical columns to use indicator variables, so it will be interesting to see how models handle the large amount of information.

We will leave the null values as is for now. Once we have performed some exploratory data analysis, we will be better able to decide what to do with the values. Many we will be able to convert to an 'other' value.

In terms of the clustering model, we may only use the features up through the stop, so some of the null values may not matter anyway, such as the stop_outcome. We still want to have all features in the exploratory protion though.